In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os 
import json

from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import *
from data import ImageCaptionDataset, generate_batch
from model import ImageCaptionModel
from torch.utils.data import DataLoader
import torch


In [3]:
data_path = 'data'
imgs_path = "data/img"

In [4]:
captions_json = {f.replace('captions_','').replace('2014.json',''):f for f in os.listdir(data_path) if f.startswith('caption')}
captions_json

{'val': 'captions_val2014.json', 'train': 'captions_train2014.json'}

In [5]:
df_train, df_val = load_images(captions_json, imgs_path)

In [6]:
#plot_images(df_train, os.path.join(imgs_path,'train'))
#plot_images(df_val, os.path.join(imgs_path,'val'))

In [7]:
transform = get_transform()
transform

Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
    RandomCrop(size=(224, 224), padding=None)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
)

In [8]:
dataset = ImageCaptionDataset(transform=transform, 
                              df=df_train, 
                              img_path= os.path.join(imgs_path, 'train')
                              )

In [9]:
dataset.char_to_id('<pad>')

1

In [10]:
BATCH_SIZE = 4

In [11]:
train_loader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, collate_fn=generate_batch)

In [12]:
vocab_size = len(train_loader.dataset.vocab)
vocab_size

7967